# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1:

#### Use the Notebook to build the code to scrape the following Wikipedia page,
#### https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
#### in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src="po_table.png" width="50%" height="750%"></img>

#### To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

## import the necessary libraries

In [31]:
#!conda install -c conda-forge beautifulsoup4 --yes # get BeautifulSoup package

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

from bs4 import BeautifulSoup # to parse html file

print('Libraries imported.')

Libraries imported.


## get postcode data for toronto from wikipedia

In [32]:
# get data from wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# convert to beautiful soup 
soup = BeautifulSoup(source, 'lxml') ##'html.parser') #'lxml')
#print(soup)

In [33]:
# this method return the index when element is found
# and return -1 when element is not found
def find_in_list(lst, value):
    try:
        return lst.index(value)
    except:
        return -1

In [34]:
# find the table in the page
table = soup.find('table', class_="wikitable sortable")

# define lists to store values in each table column
pc=[] # list for postcode
bh=[] # list for borough
nd=[] # list for neighborhood

# loop through each row in the wiki table
for row in table.find_all("tr"):
    try:
        # get the column values in each row
        col = row.find_all("td")
        
        # rule #1: Ignore cells with a borough that is Not assigned.
        if col[1].text != "Not assigned" :
            # rule #2: when multiple neighborhood can exist in one postalcode area, 
            #          combine neignbordhood with commas
            idx = find_in_list(pc, col[0].text)
            #print (idx)
            if idx > 0:
                nd[idx] = nd[idx] + "," + col[2].text.replace('\n','') # clean column value
                
            else:
                pc.append(col[0].text)
                bh.append(col[1].text)
                # rule #3: If a cell has a borough but a 'Not assigned' neighborhood, 
                #          then the neighborhood will be the same as the borough.
                if col[2].text.replace('\n','') == "Not assigned":
                    nd.append(col[1].text)
                else:
                    nd.append(col[2].text.replace('\n','')) # clean column value
            
    except Exception as e:
        pass

# define column names    
column=['PostalCode', 'Borough', 'Neighborhood']

# create a dataframe from the data
tto=pd.DataFrame({column[0]:pc,column[1]:bh, column[2]:nd})

# display some rows in the dataframe
print(tto.head())

# save dataframe to csv file
tto.to_csv('temp_toronto_postcodes_borough_neighborhood.csv',index=None)


  PostalCode           Borough                     Neighborhood
0        M3A        North York                        Parkwoods
1        M4A        North York                 Victoria Village
2        M5A  Downtown Toronto         Harbourfront,Regent Park
3        M6A        North York  Lawrence Heights,Lawrence Manor
4        M7A      Queen's Park                     Queen's Park


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [38]:
# print the dataframe's size 
print(tto.shape)

(103, 3)


In [36]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
##from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
%pip install geocoder

Note: you may need to restart the kernel to use updated packages.


## Part 2 : Use the Geocoder package or the csv file to create the following dataframe:


<img src="Part2.png" width="50%" height="750%"></img>

#### Install geocoder

#### Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [15]:
import geocoder # import geocoder

In [30]:
# initialize your variable to None
lat_lng_coords = None

# define list to store the post code and its corresponding latitude and longitudes
lst=[]
for postal_code in pc:
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        sleep(1) 
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    lst.append(print("{},{},{}".format(postal_code,latitude,longitude)))

import csv
#write this information to a csv file
with open('Geospatial_Coordinates.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['Postal Code','Latitude','Longitude'])
    for row in lst:
        r = row.split(',')
        csv_writer.writerow([r[0],r[1],r[2]])


#### !! unable to extract data from geocoder, so using 'Geospatial_Coordinates.csv'

In [54]:
# coordinates dataframe
coordDf = pd.read_csv('Geospatial_Coordinates.csv')
coordDf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Rename the common column in coordinate data frame

In [55]:
coordDf.rename(columns={"Postal Code": "PostalCode"}, inplace=True) ## 
coordDf.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [56]:
# post code, borough and neighborhood
tto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


#### Merge the two dataframes

In [57]:
mg = pd.merge(tto, coordDf, on='PostalCode' )
mg.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Total records

In [58]:
mg.shape

(103, 5)